#### Imports

In [1]:
import textwrap
import logging
import os
import sys

# Disable logging for the httpx library
logging.getLogger("httpx").disabled = True


In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [3]:
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import (
    OTLPSpanExporter as HTTPSpanExporter,
)
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

In [4]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event

In [5]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event


class PrepEvent(Event):
    pass


class InputEvent(Event):
    input: list[ChatMessage]


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

#### Agent class

In [72]:
from typing import Any, List

from llama_index.core.agent.react import ReActChatFormatter, ReActOutputParser
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.llms.openai import OpenAI


class ReActAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: LLM | None = None,
        tools: list[BaseTool] | None = None,
        extra_context: str | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.formatter = ReActChatFormatter(context=extra_context or "")
        self.output_parser = ReActOutputParser()
        self.sources = []

    @step(pass_context=True)
    async def new_user_msg(self, ctx: Context, ev: StartEvent) -> PrepEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # clear current reasoning
        ctx.data["current_reasoning"] = []

        return PrepEvent()

    @step(pass_context=True)
    async def prepare_chat_history(
        self, ctx: Context, ev: PrepEvent
    ) -> InputEvent:
        # get chat history
        chat_history = self.memory.get()
        current_reasoning = ctx.data.get("current_reasoning", [])
        llm_input = self.formatter.format(
            self.tools, chat_history, current_reasoning=current_reasoning
        )
        return InputEvent(input=llm_input)

    @step(pass_context=True)
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input

        response = await self.llm.achat(chat_history)

        try:
            reasoning_step = self.output_parser.parse(response.message.content)
            ctx.data.get("current_reasoning", []).append(reasoning_step)
            if reasoning_step.is_done:
                self.memory.put(
                    ChatMessage(
                        role="assistant", content=reasoning_step.response
                    )
                )
                return StopEvent(
                    result={
                        "response": reasoning_step.response,
                        "sources": [*self.sources],
                        "reasoning": ctx.data.get("current_reasoning", []),
                    }
                )
            elif isinstance(reasoning_step, ActionReasoningStep):
                tool_name = reasoning_step.action
                tool_args = reasoning_step.action_input
                return ToolCallEvent(
                    tool_calls=[
                        ToolSelection(
                            tool_id="fake",
                            tool_name=tool_name,
                            tool_kwargs=tool_args,
                        )
                    ]
                )
        except Exception as e:
            ctx.data.get("current_reasoning", []).append(
                ObservationReasoningStep(
                    observation=f"There was an error in parsing my reasoning: {e}"
                )
            )

        # if no tool calls or final response, iterate again
        return PrepEvent()

    @step(pass_context=True)
    async def handle_tool_calls(
        self, ctx: Context, ev: ToolCallEvent
    ) -> PrepEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.name: tool for tool in self.tools}

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            if not tool:
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(
                        observation=f"Tool {tool_call.tool_name} does not exist"
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(observation=str(tool_output))
                )
            except Exception as e:
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(
                        observation=f"Error calling tool {tool.metadata.name}: {str(e)}"
                    )
                )

        # prep the next iteration
        return PrepEvent()

#### Basic rag query engine

In [17]:
import data_pull_and_prep.utils as utils
import data_pull_and_prep.data_preparation as data_prep
import textwrap
import basic_rag.rag as rag
from basic_rag.utils import aRetrieveAndAnswer, aRetrieveAndAnswer, RetrieveAndAnswer

In [8]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

custom_chunking_obj = data_prep.CreateCustomTextChunks(transcription_with_char_timestamps)
text_chunks_with_timestamps = custom_chunking_obj.create_custom_text_chunks()

custom_ingestion_obj = rag.CustomRAG(
              index_name="ivanka-08-28-via-class",
              text_chunks_with_timestamps=text_chunks_with_timestamps
              )

await custom_ingestion_obj.create_text_nodes_and_add_to_vector_store()

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/rishikeshdhayarkar/rag-audio-indexing/rag-audio-env/lib/python3.12/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Upserted vectors: 100%|██████████| 42/42 [00:02<00:00, 18.55it/s]


In [66]:
def basic_rag_query_engine(input: str):
    raa = RetrieveAndAnswer(ingestion_obj=custom_ingestion_obj)
    return raa.answer(input)

In [ ]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"
print(textwrap.fill(basic_rag_query_engine(query_str), 80))

#### Agent setup

In [73]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.tools import FunctionTool

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

basic_rag_tool = FunctionTool(
    fn=basic_rag_query_engine,
    metadata=ToolMetadata(
        name="basic_rag_query_engine",
        description="Provides information about the Ivanka Trump podcast using a basic RAG approach. Use a plain text question as input to the tool."
    )
)

query_engine_tools = [
    basic_rag_tool
]

In [74]:
agent = ReActAgent(
    llm=OpenAI(model="gpt-3.5-turbo"), tools=query_engine_tools,
    timeout=120,
    verbose=False,
    extra_context="Please use the query engine tool(basic_rag_query_engine) to answer the questions.",
)

ret = await agent.run(input="Hello!")

#### Querying

In [75]:
query_str_suffix = " Please use the query engine tools to answer the questions."

In [ ]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"+query_str_suffix
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

In [ ]:
query_str = "What are Ivanka Trump's thoughts on music?"+query_str_suffix
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

In [ ]:
query_str = "There must be a music related stuff in the context. Give me more details on that."+query_str_suffix
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

In [ ]:
query_str = "describe the incident with kim kardashian. Something about prisons"+query_str_suffix
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

In [ ]:
query_str = "What are some architectural projects that Ivanka Trump has worked on?"+query_str_suffix
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

In [15]:
query_str = "Describe the impact of NYC on Ivanka Trump's life"
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
New York City has had a significant impact on Ivanka Trump's life. She has been
involved in various architectural projects in the city, such as the Trump Tower.
Additionally, NYC is a hub for business, fashion, and culture, all of which have
influenced Ivanka Trump's career and personal life.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response="New York City has had a
significant impact on Ivanka Trump's life. She has been involved in various
architectural projects in the city, such as the Trump Tower. Additionally, NYC
is a hub for business, fashion, and culture, all of which have influenced Ivanka
Trump's career and personal life.", is_streaming=False)]


In [14]:
query_str = "What buildings has Ivanka Trump worked on?"
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
Ivanka Trump has worked on various architectural projects, including the Trump
International Hotel in Washington, D.C., the Trump Tower in New York City, and
the renovation of the Old Post Office Pavilion in Washington, D.C.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response='Ivanka Trump has worked on various
architectural projects, including the Trump International Hotel in Washington,
D.C., the Trump Tower in New York City, and the renovation of the Old Post
Office Pavilion in Washington, D.C.', is_streaming=False)]


In [15]:
query_str = "What does Ivanka Trump say about her children and husband?"
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
Ivanka Trump mentioned in the podcast that her children bring her immense joy
and fulfillment. She also expressed admiration and gratitude for her husband,
Jared Kushner, highlighting his intelligence, work ethic, and support in both
personal and professional aspects of her life.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response='Ivanka Trump mentioned in the
podcast that her children bring her immense joy and fulfillment. She also
expressed admiration and gratitude for her husband, Jared Kushner, highlighting
his intelligence, work ethic, and support in both personal and professional
aspects of her life.', is_streaming=False)]


In [16]:
query_str = "Describe the type of work Ivanka Trump did as a senior advisor to the president"
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
Ivanka Trump served as a senior advisor to the president, focusing on various
policy initiatives such as workforce development, job creation, economic
empowerment, and women's entrepreneurship. She was involved in advocating for
paid family leave, workforce training programs, and initiatives to support women
in business. Additionally, Ivanka Trump worked on criminal justice reform
efforts and other policy issues during her time in the White House.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought='(Implicit) I can answer without any more
tools!', response="Ivanka Trump served as a senior advisor to the president,
focusing on various policy initiatives such as workforce development, job
creation, economic empowerment, and women's entrepreneurship. She was involved
in advocating for paid family leave, workfo

In [17]:
query_str = "Describe the type of work Ivanka Trump did on taxes"
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
Ivanka Trump worked on tax-related initiatives during her time as a senior
advisor to the president, focusing on issues such as tax reform, tax credits for
families, and economic policies aimed at promoting growth and job creation. She
was involved in advocating for tax policies that would benefit American families
and businesses.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response='Ivanka Trump worked on tax-related
initiatives during her time as a senior advisor to the president, focusing on
issues such as tax reform, tax credits for families, and economic policies aimed
at promoting growth and job creation. She was involved in advocating for tax
policies that would benefit American families and businesses.',
is_s

In [18]:
query_str = """What unique considerations and complexities were involved in the renovation of the old post 
office building, particularly in terms of layout, room configurations, and preserving the building's historic exterior?"""
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
The renovation of the Old Post Office Building, which was transformed into the
Trump International Hotel, involved unique considerations and complexities.
These included preserving the historic exterior of the building while
reconfiguring the interior layout to accommodate modern amenities and
functionality. The challenge was to maintain the architectural integrity of the
historic structure while adapting it to serve a new purpose as a luxury hotel.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response='The renovation of the Old Post
Office Building, which was transformed into the Trump International Hotel,
involved unique considerations and complexities. These included preserving the
historic exterior of the building

In [19]:
query_str = """How did Ivanka Trump's children, particularly her son Theo, contribute to her sense of grounding and joy during her time in Washington, D.C.?"""
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
Ivanka Trump mentioned in the podcast that her children, especially her son
Theo, brought her a sense of grounding and joy during her time in Washington,
D.C. Their presence provided her with a sense of normalcy and balance amidst the
intense environment of politics and public service. Theo's innocence and love
helped Ivanka Trump stay connected to what truly mattered to her, beyond her
professional responsibilities.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response="Ivanka Trump mentioned in the
podcast that her children, especially her son Theo, brought her a sense of
grounding and joy during her time in Washington, D.C. Their presence provided
her with a sense of normalcy and balance amidst the intense environm

In [20]:
query_str = """What does Ivanka Trump like to do in her free time? What are her hobbies and interests?"""
ret = await agent.run(input=query_str)
print("-"*100)
print(textwrap.fill(ret["response"], 80))
print("-"*100)
print(textwrap.fill(str(ret["reasoning"]), 80))

----------------------------------------------------------------------------------------------------
In her free time, Ivanka Trump enjoys playing with her children, spending time
with her family, and engaging in activities like cooking, reading, and playing
the guitar. She also mentioned in the podcast that she values moments of
relaxation and self-care to recharge and maintain a healthy work-life balance.
----------------------------------------------------------------------------------------------------
[ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer", response='In her free time, Ivanka Trump
enjoys playing with her children, spending time with her family, and engaging in
activities like cooking, reading, and playing the guitar. She also mentioned in
the podcast that she values moments of relaxation and self-care to recharge and
maintain a healthy work-life balance.', is_streaming=False)]


The revenue of Uber in 2021 was $17.455 billion.


In [19]:
print(textwrap.fill(str(ret["sources"]), 80))

[ToolOutput(content='$17,455 million.', tool_name='uber_10k',
raw_input={'input': 'What was the revenue of Uber in 2021?'},
raw_output=Response(response='$17,455 million.', source_nodes=[NodeWithScore(nod
e=TextNode(id_='c8d907bf-18c2-4486-a08b-dbc70b81d4ae', embedding=None,
metadata={'page_label': '57', 'file_name': 'uber_2021.pdf', 'file_path':
'data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483,
'creation_date': '2024-08-22', 'last_modified_date': '2024-08-22'},
excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size',
'creation_date', 'last_modified_date', 'last_accessed_date'],
excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size',
'creation_date', 'last_modified_date', 'last_accessed_date'],
relationships={<NodeRelationship.SOURCE: '1'>:
RelatedNodeInfo(node_id='98731e04-f478-48bb-aca9-e7599235c3b6',
node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '57', 'file_name':
'uber_2021.pdf', 'file_path': 'data/10k/uber_2021.p

In [20]:
print(textwrap.fill(str(ret["reasoning"]), 80))

[ActionReasoningStep(thought='The current language of the user is: English. I
need to use a tool to help me answer the question.', action='uber_10k',
action_input={'input': 'What was the revenue of Uber in 2021?'}),
ObservationReasoningStep(observation='$17,455 million.', return_direct=False),
ResponseReasoningStep(thought="I can answer without using any more tools. I'll
use the user's language to answer.", response='The revenue of Uber in 2021 was
$17,455 million.', is_streaming=False)]


In [27]:
ret = await agent.run(input="How does the revenue of Uber compare to Lyft in 2021? Give me the answer in Billions")

Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Step handle_llm_input produced event StopEvent


In [28]:
print(textwrap.fill(str(ret["reasoning"]), 80))

[ResponseReasoningStep(thought='(Implicit) I can answer without any more
tools!', response="In 2021, Uber's revenue was approximately $17.46 billion,
while Lyft's revenue was about $3.21 billion. Therefore, Uber's revenue was
significantly higher than Lyft's, with Uber earning roughly 5.44 times more than
Lyft.", is_streaming=False)]
